In [1]:
import pulp
import numpy as np
import time


### 1. Parameters definition

In [2]:
# Original vector
v = [3.2, -1.4, 2.5, -0.9, 1.8, -3.7, 0.0, 4.0, 2.2, -1.3]

# 3-bit quantization: 8 levels uniformly distributed from -4 to 4
num_bits = 3
num_levels = 2 ** num_bits  # 8 levels
min_val = -4
max_val = 4

# Generate quantization levels
Q = np.linspace(min_val, max_val, num=num_levels)
print("Quantization Levels (Q):", Q)

Quantization Levels (Q): [-4.         -2.85714286 -1.71428571 -0.57142857  0.57142857  1.71428571
  2.85714286  4.        ]


### 2. Optimization problem

In [3]:
def vector_quantization_optimization(v, Q):
    # Initialize the optimization problem
    prob = pulp.LpProblem("Vector_Quantization", pulp.LpMinimize)

    vector_indices = range(len(v))         
    quant_levels_indices = range(len(Q))  

    # Define decision variables
    x_vars = pulp.LpVariable.dicts("x",
                                    ((i, j) for i in vector_indices for j in quant_levels_indices),
                                    cat='Binary') 

    # Define the objective function: minimize the sum of L2 norms
    prob += pulp.lpSum([x_vars[(i, j)] * (v[i] - Q[j]) ** 2
                        for i in vector_indices
                        for j in quant_levels_indices]), "Objective"

    # Add constraints: Each vector element is assigned to exactly one quantization level
    for i in vector_indices:
        prob += pulp.lpSum([x_vars[(i, j)] for j in quant_levels_indices]) == 1, f"Assignment_{i}"
    return prob, x_vars

### 3. Solution of the optimization problem

In [4]:
def Lp_quantization_assignment(v, Q):
    # Solve the optimization problem
    prob, x_vars = vector_quantization_optimization(v, Q)
    prob.solve()
    
    # Check if the solution is optimal
    if pulp.LpStatus[prob.status] == "Optimal":
        # Extract quantized vector
        quant_levels_indices = range(len(Q))
        vector_indices = range(len(v))
        
        q = []
        for i in vector_indices:
            for j in quant_levels_indices:
                if pulp.value(x_vars[(i, j)]) == 1:
                    q.append(Q[j])
                    break

        return q
    else:
        raise ValueError("No optimal solution found!")

# Quantize the vector
q= Lp_quantization_assignment(v, Q)
print("Original Vector:", v)
print("Quantized Vector:", q)

# Calculate the L2 norm
l2_norm = np.sqrt(sum((vi - qi) ** 2 for vi, qi in zip(v, q)))
print("L2 Norm:", l2_norm)


Original Vector: [3.2, -1.4, 2.5, -0.9, 1.8, -3.7, 0.0, 4.0, 2.2, -1.3]
Quantized Vector: [2.8571428571428568, -1.7142857142857144, 2.8571428571428568, -0.5714285714285716, 1.7142857142857135, -4.0, -0.5714285714285716, 4.0, 1.7142857142857135, -1.7142857142857144]
L2 Norm: 1.1328130058056582


### 4. Heuristic algorithm

In [5]:
def heuristic_quantization_assignment(v, Q):
    """
    Assign each element in vector v to the nearest quantization level in Q.
    
    Parameters:
    v (list or array): The vector to quantize.
    Q (list or array): Quantization levels 
    
    Returns:
    list: Quantized vector with elements from Q.
    """
    # Step 1: Build the boundaries B
    B = [(Q[j] + Q[j + 1]) / 2 for j in range(len(Q) - 1)]
    B = [-np.inf] + B + [np.inf]
    
    # Step 2: Quantize each element in v
    quantized_v = []
    for vi in v:
        j = find_largest_index(vi, B)
        quantized_v.append(Q[j])
    
    return quantized_v

def find_largest_index(vi, B):
    """
    Find the largest index j for which B[j] ≤ vi < B[j + 1].
    
    Parameters:
    vi (float): The value to find the index for.
    B (list): The boundaries.
    
    Returns:
    int: The index j such that B[j] ≤ vi < B[j + 1].
    """
    j = 0
    while vi > B[j + 1]:
        j += 1
    return j

quantized_v = heuristic_quantization_assignment(v, Q)
print("Quantized vector:", quantized_v)

# Calculate the L2 norm
l2_norm = np.sqrt(sum((vi - qi) ** 2 for vi, qi in zip(v, quantized_v)))
print("L2 Norm:", l2_norm)


Quantized vector: [2.8571428571428568, -1.7142857142857144, 2.8571428571428568, -0.5714285714285716, 1.7142857142857135, -4.0, 0.5714285714285712, 4.0, 1.7142857142857135, -1.7142857142857144]
L2 Norm: 1.132813005805658


5. Comparison of the heuristic algorithm with the Lp solver

In [ ]:
num_trials = 10
Lp_times = []
Lp_L2_norms = []
heuristic_times = []
heuristic_L2_norms = []

# Run the experiment with different seeds and collect timing data
for seed in range(num_trials):
    np.random.seed(seed)
    v_large = np.random.normal(loc=0, scale=5, size=100000)

    # Time the optimization-based quantization
    start_time = time.time()
    quantized_v_opt = Lp_quantization_assignment(v_large, Q)
    end_time = time.time()
    Lp_times.append(end_time - start_time)
    lp_l2_norm = np.sqrt(sum((vi - qi) ** 2 for vi, qi in zip(v_large, quantized_v_opt)))
    Lp_L2_norms.append(lp_l2_norm)

    # Time the heuristic quantization
    start_time = time.time()
    quantized_v_heuristic = heuristic_quantization_assignment(v_large, Q)
    end_time = time.time()
    heuristic_times.append(end_time - start_time)
    heuristic_l2_norm = np.sqrt(sum((vi - qi) ** 2 for vi, qi in zip(v_large, quantized_v_heuristic)))
    heuristic_L2_norms.append(heuristic_l2_norm)
# Calculate speedup for each instance
instance_speedups = [Lp_time / heuristic_time for Lp_time, heuristic_time in zip(Lp_times, heuristic_times)]

# Calculate average times
avg_Lp_time = np.mean(Lp_times)
avg_heuristic_time = np.mean(heuristic_times)
avg_Lp_L2_norm = np.mean(Lp_L2_norms)
avg_heuristic_L2_norm = np.mean(heuristic_L2_norms)
avg_speedup = np.mean(instance_speedups)

# Display results
print(f"Average Optimization-based quantization time: {avg_Lp_time:.4f} seconds")
print(f"Average Heuristic quantization time: {avg_heuristic_time:.4f} seconds")
print(f"Average Speedup: {avg_speedup:.2f} times")
print(f"Average Optimization-based quantization L2 norm: {avg_Lp_L2_norm:.4f}")
print(f"Average Heuristic quantization L2 norm: {avg_heuristic_L2_norm:.4f}")


Average Optimization-based quantization time: 22.7139 seconds
Average Heuristic quantization time: 0.0421 seconds
Average Speedup: 539.55 times
Average Optimization-based quantization L2 norm: 763.6547
Average Heuristic quantization L2 norm: 763.6547
Average Speedup: 540.4701932236557
